# Mid-term Report. Hyerin Kim (student ID:20185290)


## Data files
<ul>
    <li> The genome sequence of Covid-19: '../data/2019nCoV_genomes.2020_04_14.clean.fa'
    <li> reference sequnece(DNA) : '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
    <li> reference seqeunce(protein) : '../data/GCF_009858895.2_ASM985889v3_protein.faa'
    <li> reference information : '../data/GCF_009858895.2_ASM985889v3_genomic.gff'
</ul>
    
## Procedures
<ol>
    <li> isolate genome seqeunces collected from US
    <li> make protein sequence from 2019nCoV_genomes.2020_04_14.clean.fa by transeq
    <li> blastp with query: reference seqeunce(protein), db: transeq result
    <li> isolate protein sequences
    <li> Do Multiple Sequence Alignment & Phylogeny Construction
</ol>

## Questions
<ol>
    <li> Using multiple sequence analysis and phylogenetic tree reconstruction, can you estimate the propagation path of SARS-CoV-2 in the US? Justify your idea with the data.
    <li> Based on the number of new mutations & the collection time, can you estimate the “mutation rate”? (how long does it take to make a new mutation?)
    <li> Is there any specific gene having higher rate of mutation? Can you explain why?
</ol>

In [1]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# isolate genomes by country
def isolate(genomes,country):
    rv = dict()
    for k, v in genomes.items():
        k = k.replace("|", "@")
        if country in k:
            rv[k] = v
        
    return rv

In [3]:
# the function to write fasta file
def write_fasta(tmp_filename, gene_list):
    file = open(tmp_filename,"w")
    for key, values in gene_list.items():
        file.write(">"+key+"\n"+values+"\n")
    file.close()

In [4]:
# isolate SARS-CoV2 genomes from US
COVID19 = read_fasta('2019nCoV_genomes.2020_04_14.clean.fa')

COVID19_USA = isolate(COVID19, 'USA')

#remove same genomic seqeunce 
print(len(COVID19_USA))
COVID19_USA_clear = dict()
for k, v in COVID19_USA.items():
    if v not in COVID19_USA_clear.values():
        COVID19_USA_clear[k] = v
print(len(COVID19_USA_clear))


write_fasta('COVID19_USA.fa',COVID19_USA_clear)

647
589


In [5]:
# make protein by transeq
'''
type on Ubuntu

muscle -in COVID19_USA.fa -out COVID19_USA_muscle.fa
transeq --sequence COVID19_USA.fa --outseq COVID19_USA_prot.fa -frame 6

makeblastdb -dbtype prot -in COVID19_USA_prot.fa -out COVID19_USA_prot_db
blastp -db COVID19_USA_prot_db -query ../data/GCF_009858895.2_ASM985889v3_protein.faa -outfmt 7 -out COVID19_USA_blastp.tbl -max_target_seqs 600

phmmer --dmblout COVID19_USA_hmm ../data/GCF_009858895.2_ASM985889v3_protein.faa COVID19_USA_prot.fa
'''

'\ntype on Ubuntu\n\nmuscle -in COVID19_USA.fa -out COVID19_USA_muscle.fa\ntranseq --sequence COVID19_USA.fa --outseq COVID19_USA_prot.fa -frame 6\n\nmakeblastdb -dbtype prot -in COVID19_USA_prot.fa -out COVID19_USA_prot_db\nblastp -db COVID19_USA_prot_db -query ../data/GCF_009858895.2_ASM985889v3_protein.faa -outfmt 7 -out COVID19_USA_blastp.tbl -max_target_seqs 600\n\nphmmer --dmblout COVID19_USA_hmm ../data/GCF_009858895.2_ASM985889v3_protein.faa COVID19_USA_prot.fa\n'

In [6]:
#the function to read blastP result
def blastP(tmp_filename, protein, length):
    result = dict()
    info = list()
    f = open(tmp_filename,'r')
    for line in f:
        if not line.startswith('#') and line.startswith(protein):
            if line.split()[1] not in result.keys() and line.split()[6] == '1' and line.split()[7] == length:
                # info = [% identity, s. start, s. end], -1 is due to str starts 0 not 1
                info = [float(line.split()[2]),int(line.split()[8])-1,int(line.split()[9])-1]
                # result = {subject:info}            
                result[line.split()[1]] = info

        info = list()
    f.close()
    return result

In [7]:
# the function to extract DNA or Protein seqeunce
def extract(tmp_filename, result):
    f = open(tmp_filename,'r')
    ext_seq = dict()
    whole = read_fasta(tmp_filename)
    for k, v in result.items():
        for name, seq in whole.items():
            if k == name:
                #save the blast result in info = [%identity, s.start, s.end] v[1]=s.start, v[2]=s.end
                ext_seq[name] = seq[v[1]:v[2]]
    f.close()
    return ext_seq
        
    

In [21]:
#the fucntion which select representative seqeunce in dictionary
def selection(gene_list):
    gene_select = dict()
    for k, v in gene_list.items():
        # repetitive seqeunce are removed
        if v not in gene_select.values():
            gene_select[k] = v
            
    return gene_select

In [26]:
#isolate orf1ab polyprotein seqeunce from blastp result
types = dict()
COVID19_USA_YP_009724389 = blastP('COVID19_USA_blastp.tbl','YP_009724389.1','4401')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724389)
example_rep = selection(example)
print(date(example))
print(len(COVID19_USA_YP_009724389))
print(len(example))
print(len(example_rep))
types['orf1ab polyprotein'] = len(example_rep)
#remove same 
write_fasta('orf1ab_polyprotein.fas',example_rep)

{'MESLVPGFNE': ['MT326179@USA@2020-03-19_1'], 'MEXXVPGFNE': ['MT326144@USA@2020-03-20_2'], 'LESLVPGFNE': ['MT293206@USA_WA@2020-03-30_3'], 'MESXVPGFNE': ['MT263451@USA_WA@2020-03-24_1'], 'MESLVPGFXX': ['MT326163@USA@2020-03-21_2'], 'MESLVPGFNX': ['MT263455@USA_WA@2020-03-24_2'], 'MXSLVPGXNE': ['MT246465@USA_WA@2020-03-13_3'], 'MESLXPGFNE': ['MT293180@USA_WA@2020-03-28_1']}
513
513
149


In [28]:
#isolate surface glycoprotein seqeunce from blastp result

COVID19_USA_YP_009724390 = blastP('COVID19_USA_blastp.tbl','YP_009724390.1','1273')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724390)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724390))
print(len(example))
print(len(example_rep))
types['surface glycoprotein'] = len(example_rep)
#remove same 
write_fasta('surface_glycoprotein.fas',example_rep)

587
587
87


In [29]:
#isolate ORF3a protein seqeunce from blastp result

COVID19_USA_YP_009724391 = blastP('COVID19_USA_blastp.tbl','YP_009724391.1','275')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724391)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724391))
print(len(example))
print(len(example_rep))
types['ORF3a protein'] = len(example_rep)
#remove same 
write_fasta('ORF3a_protein.fas',example_rep)

587
587
44


In [30]:
#isolate envelope protein seqeunce from blastp result

COVID19_USA_YP_009724392 = blastP('COVID19_USA_blastp.tbl','YP_009724392.1','75')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724392)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724392))
print(len(example))
print(len(example_rep))
types['envelope protein'] = len(example_rep)
#remove same 
write_fasta('envelope_protein.fas',example_rep)

581
581
15


In [31]:
#isolate membrane glycoprotein seqeunce from blastp result

COVID19_USA_YP_009724393 = blastP('COVID19_USA_blastp.tbl','YP_009724393.1','222')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724393)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724393))
print(len(example))
print(len(example_rep))
types['membrane glycoprotein'] = len(example_rep)
#remove same 
write_fasta('membrane_glycoprotein.fas',example_rep)

578
578
31


In [32]:
#isolate ORF6 protein seqeunce from blastp result

COVID19_USA_YP_009724394 = blastP('COVID19_USA_blastp.tbl','YP_009724394.1','61')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724394)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724394))
print(len(example))
print(len(example_rep))
types['ORF6 protein'] = len(example_rep)
#remove same 
write_fasta('ORF6_protein.fas',example_rep)

584
584
12


In [33]:
#isolate ORF7a protein seqeunce from blastp result

COVID19_USA_YP_009724395 = blastP('COVID19_USA_blastp.tbl','YP_009724395.1','121')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724395)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724395))
print(len(example))
print(len(example_rep))
types['ORF7a protein'] = len(example_rep)
#remove same 
write_fasta('ORF7a_protein.fas',example_rep)

587
587
13


In [34]:
#isolate ORF8 protein seqeunce from blastp result

COVID19_USA_YP_009724396 = blastP('COVID19_USA_blastp.tbl','YP_009724396.1','121')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724396)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724396))
print(len(example))
print(len(example_rep))
types['ORF8 protein'] = len(example_rep)
#remove same 
write_fasta('ORF8_protein.fas',example_rep)

588
588
12


In [35]:
#isolate nucleocapsid phosphoprotein seqeunce from blastp result

COVID19_USA_YP_009724397 = blastP('COVID19_USA_blastp.tbl','YP_009724397.2','419')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009724397)
example_rep = selection(example)
print(len(COVID19_USA_YP_009724397))
print(len(example))
print(len(example_rep))
types['nucleocapsid phosphoprotein'] = len(example_rep)
#remove same 
write_fasta('nucleocapsid_phosphoprotein.fas',example_rep)

589
589
37


In [36]:
#isolate ORF10 protein seqeunce from blastp result

COVID19_USA_YP_009725255 = blastP('COVID19_USA_blastp.tbl','YP_009725255.1','38')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009725255)
example_rep = selection(example)
print(len(COVID19_USA_YP_009725255))
print(len(example))
print(len(example_rep))
types['ORF10 protein'] = len(example_rep)
#remove same 
write_fasta('ORF10_protein.fas',example_rep)

589
589
4


In [37]:
#isolate orf1a polyprotein seqeunce from blastp result

COVID19_USA_YP_009725295 = blastP('COVID19_USA_blastp.tbl','YP_009725295.1','4405')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009725295)
example_rep = selection(example)
print(len(COVID19_USA_YP_009725295))
print(len(example))
print(len(example_rep))
types['orf1a polyprotein'] = len(example_rep)
#remove same 
write_fasta('orf1a_polyprotein.fas',example_rep)

513
513
149


In [26]:
#isolate ORF7b seqeunce from blastp result

COVID19_USA_YP_009725296 = blastP('COVID19_USA_blastp.tbl','YP_009725296.1','43')
example = extract('COVID19_USA_prot.fa',COVID19_USA_YP_009725296)
example_rep = selection(example)
print(len(COVID19_USA_YP_009725296))
print(len(example))
print(len(example_rep))
types['ORF7b'] = len(example_rep)
#remove same 
write_fasta('ORF7b.fas',example_rep)

589
589
3


In [38]:
print(types)

{'orf1ab polyprotein': 149, 'surface glycoprotein': 87, 'ORF3a protein': 44, 'envelope protein': 15, 'membrane glycoprotein': 31, 'ORF6 protein': 12, 'ORF7a protein': 13, 'ORF8 protein': 12, 'nucleocapsid phosphoprotein': 37, 'ORF10 protein': 4, 'orf1a polyprotein': 149}
